In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.6     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [22]:
atp_data <- read_csv("https://raw.githubusercontent.com/JianyiC/group54_work/main/atp2017-2019.csv") |>
  select(winner_age, loser_age, winner_rank, loser_rank, winner_rank_points, loser_rank_points) |> 
  na.omit()
atp_data

New names:
• `` -> `...1`
Rows: 6866 Columns: 50
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (16): tourney_id, tourney_name, surface, tourney_level, winner_seed, win...
dbl (34): ...1, draw_size, tourney_date, match_num, winner_id, winner_ht, wi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


winner_age,loser_age,winner_rank,loser_rank,winner_rank_points,loser_rank_points
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29.00479,22.88569,9,16,3590,1977
22.88569,33.70568,16,239,1977,200
29.00479,31.88227,9,40,3590,1050
⋮,⋮,⋮,⋮,⋮,⋮
32.59411,33.68925,15,76,2320,667
26.95414,32.59411,7,15,3775,2320
23.73990,33.68925,18,76,2235,667


In [38]:
lose <- rep(c("lose"), times = nrow(atp_data))

atp_lose <- cbind(atp_data, lose) |>
  select(lose, loser_age, loser_rank, loser_rank_points)

colnames(atp_lose) <- c("result", "age","rank", "points")
atp_data_lose

result,age,rank,NA
<chr>,<dbl>,<dbl>,<dbl>
lose,22.88569,16,1977
lose,33.70568,239,200
lose,31.88227,40,1050
⋮,⋮,⋮,⋮
lose,33.68925,76,667
lose,32.59411,15,2320
lose,33.68925,76,667


In [36]:
win <- rep(c("win"), times = nrow(atp_data))

atp_win <- cbind(atp_data, win) |>
  select(win, winner_age, winner_rank, winner_rank_points)

colnames(atp_win) <- c("result", "age","rank", "points")

atp_data_win

result,age,rank,NA
<chr>,<dbl>,<dbl>,<dbl>
win,29.00479,9,3590
win,22.88569,16,1977
win,29.00479,9,3590
⋮,⋮,⋮,⋮
win,32.59411,15,2320
win,26.95414,7,3775
win,23.73990,18,2235


In [39]:
data_split <- initial_split(data, prop = 0.75, strata = result)  
atp_train <- training(data_split)   
atp_test <- testing(data_split)

atp_train
atp_test

,result,age,rank,first_svpt_won
,<fct>,<dbl>,<dbl>,<dbl>
6771,lose,22.88569,16,34
6772,lose,33.70568,239,36
6775,lose,28.01095,18,46
⋮,⋮,⋮,⋮,⋮
6768,win,32.59411,15,31
6769,win,26.95414,7,47
6770,win,23.73990,18,34


,result,age,rank,first_svpt_won
,<fct>,<dbl>,<dbl>,<dbl>
1,win,29.00479,9,31
2,win,22.88569,16,28
3,win,29.00479,9,26
⋮,⋮,⋮,⋮,⋮
13527,lose,29.12526,5,37
13531,lose,24.19439,4,14
13532,lose,24.19439,4,35
